# Semantic Memory

In [1]:
# %pip install -U langmem langgraph

In [2]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.store.memory import InMemoryStore

checkpointer = InMemorySaver()
store = InMemoryStore()

In [10]:
from langchain_groq import ChatGroq

# Load free model from Groq (you need a Groq API key)
llm = ChatGroq(
    api_key="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",
    model_name="mixtral-8x7b-32768",  # or "", etc.
)

In [12]:
agent = create_react_agent(llm, tools=[], store=store, checkpointer=checkpointer)

In [13]:
def chat(agent, txt, thread_id):
    result_state = agent.invoke({"messages": [{"role": "user", "content": txt}]}, config={"configurable": {"thread_id": thread_id}})
    return result_state["messages"][-1].content

In [14]:
thread_1 = "thread-1"
chat(agent, "Hi there, I'm training for a half marathon in 2 months - could you propose a daily training plan to prepare?", thread_1)

APIConnectionError: Connection error.

In [ ]:
chat(agent, "Nice! Wish me luck!", thread_1)

"You got this! I wish you the best of luck on your half marathon training and the big day itself! May your runs be smooth, your hills be conquered, and your finish line be a joyful celebration of your hard work and dedication.\n\nRemember to stay focused, stay positive, and stay fueled. And don't forget to celebrate your small victories along the way, because every step you take brings you closer to your goal.\n\nYou've got this!"

In [ ]:
thread_2 = "thread-2"
chat(agent, "Nice! Oh thank you! It'll be hard.", thread_2)

"I think there might have been some misunderstanding! I didn't say anything prior to your message, so I'm not sure what you're thanking me for or referring to. If you'd like to talk about something specific, I'm here to listen and help if I can!"

## Adding Memory

We'll use regular tools.

In [ ]:
%pip install sentence-transformers langmem


Note: you may need to restart the kernel to use updated packages.


In [ ]:
from langmem import create_manage_memory_tool, create_search_memory_tool

store = InMemoryStore(
    index={
        "dims": 1536,
        "embed": "openai:text-embedding-3-small"
    }
)

namespace = ("agent_memories",)
memory_tools = [
    create_manage_memory_tool(namespace),
    create_search_memory_tool(namespace)
]
checkpointer = InMemorySaver()
agent = create_react_agent(llm, tools=memory_tools, store=store, checkpointer=checkpointer)

In [ ]:
from torch import cuda
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

embed_model = HuggingFaceEmbeddings(
    model_name=embed_model_id,
    model_kwargs={'device': device},
    encode_kwargs={'device': device, 'batch_size': 32},
)

In [ ]:
# from langchain.agents import create_react_agent
# from langchain.llms import HuggingFaceHub  # or use your preferred LLM

# # Example LLM (replace with your own)
# from langchain_community.llms import HuggingFacePipeline
# from transformers import pipeline
# from langchain.prompts import PromptTemplate
# from langchain.chains import LLMChain
# # Step 1: Define fitness trainer prompt template
# fitness_prompt = PromptTemplate.from_template("""
# You are a professional AI fitness coach.

# You help users by understanding their current fitness goals, routines, and body status.
# You respond with empathy, scientific reasoning, and personalized plans.

# User: {input}
# """)

# # Step 2: Load LLM
# llm_pipeline = pipeline("text-generation", model="tiiuae/falcon-7b-instruct", max_new_tokens=512, do_sample=True)
# llm = HuggingFacePipeline(pipeline=llm_pipeline)

# # Wrap it in LLMChain with prompt
# fitness_chain = LLMChain(llm=llm, prompt=fitness_prompt)

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/17.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs.hf.co/repos/fb/9d/fb9d2cbe2e913461ae463cd35cb5a43ad1c5115ba9e913382f147e2f1ea66f3d/b4515a8e32f81ab4fef65e10891fdb188bc45c4195620d447bc3263e132d9de5?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model-00001-of-00002.safetensors%3B+filename%3D%22model-00001-of-00002.safetensors%22%3B&Expires=1750268220&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc1MDI2ODIyMH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy9mYi85ZC9mYjlkMmNiZTJlOTEzNDYxYWU0NjNjZDM1Y2I1YTQzYWQxYzUxMTViYTllOTEzMzgyZjE0N2UyZjFlYTY2ZjNkL2I0NTE1YThlMzJmODFhYjRmZWY2NWUxMDg5MWZkYjE4OGJjNDVjNDE5NTYyMGQ0NDdiYzMyNjNlMTMyZDlkZTU%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=RBikuCu6ZAIBOjZXbBp0PfFrBmUOdTLXfdOdW2usb8fPz9In6ckrj4VyzlixqYvfJfSkFzUBR84iE7ix5C%7E7pOmaWDUV5qMB-TqSR0Kwy9lryqVgnGH2Ho4%7E4MsnUffpQkpkljQ6DseGYkWC5aqAqhqiz6w63RnkV08MHdoF8C7HAf7IyC%7E-oXBIRgiengigwsiOyeYwJ%7EqTwdHc9Vs9eIvJK6UQFSPrJep3U

model-00001-of-00002.safetensors:   4%|4         | 430M/9.95G [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs.hf.co/repos/fb/9d/fb9d2cbe2e913461ae463cd35cb5a43ad1c5115ba9e913382f147e2f1ea66f3d/b4515a8e32f81ab4fef65e10891fdb188bc45c4195620d447bc3263e132d9de5?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model-00001-of-00002.safetensors%3B+filename%3D%22model-00001-of-00002.safetensors%22%3B&Expires=1750268220&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc1MDI2ODIyMH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy9mYi85ZC9mYjlkMmNiZTJlOTEzNDYxYWU0NjNjZDM1Y2I1YTQzYWQxYzUxMTViYTllOTEzMzgyZjE0N2UyZjFlYTY2ZjNkL2I0NTE1YThlMzJmODFhYjRmZWY2NWUxMDg5MWZkYjE4OGJjNDVjNDE5NTYyMGQ0NDdiYzMyNjNlMTMyZDlkZTU%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=RBikuCu6ZAIBOjZXbBp0PfFrBmUOdTLXfdOdW2usb8fPz9In6ckrj4VyzlixqYvfJfSkFzUBR84iE7ix5C%7E7pOmaWDUV5qMB-TqSR0Kwy9lryqVgnGH2Ho4%7E4MsnUffpQkpkljQ6DseGYkWC5aqAqhqiz6w63RnkV08MHdoF8C7HAf7IyC%7E-oXBIRgiengigwsiOyeYwJ%7EqTwdHc9Vs9eIvJK6UQFSPrJep3U

model-00001-of-00002.safetensors:   4%|4         | 440M/9.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

: 

In [ ]:
from langmem import create_manage_memory_tool, create_search_memory_tool
from langchain.embeddings import HuggingFaceEmbeddings

# Step 1: Use Hugging Face Embedding Model

# Step 2: Setup vector store using custom embedder

store = InMemoryStore(
    index={
        "dims": 384,
        "embed": embed_model.embed_query
    }
)
# Step 3: Create memory tools
namespace = ("agent_memories",)
memory_tools = [
    create_manage_memory_tool(namespace),
    create_search_memory_tool(namespace)
]

# Step 4: Setup checkpointer
checkpointer = InMemorySaver()

# from langchain.agents import create_react_agent, AgentExecutor
# from langchain.llms import HuggingFaceHub  # or use your preferred LLM

# # Example LLM (replace with your own)
# from langchain_community.llms import HuggingFacePipeline
# from transformers import pipeline

# llm_pipeline = pipeline("text-generation", model="tiiuae/falcon-7b-instruct", max_new_tokens=256)
# llm = HuggingFacePipeline(pipeline=llm_pipeline)

# Final Agent setup
agent = create_react_agent(llm, tools=memory_tools, store=store, checkpointer=checkpointer)
# agent_executor = AgentExecutor(agent=agent, tools=memory_tools, memory=None)



In [ ]:
thread_1 = "thread-1"
chat(agent, "Hi there, I'm training for a half marathon in 2 months - could you propose a daily training plan to prepare?", thread_1)

"Here is a sample daily training plan to help you prepare for a half marathon in 2 months:\n\nMonday:\n\n* 30-minute easy run\n* Strength training: focus on lower body (legs, glutes, calves)\n\nTuesday:\n\n* 30-minute interval training (alternate between 5-minute fast pace and 5-minute easy pace)\n* Strength training: focus on core and upper body (arms, shoulders, back)\n\nWednesday:\n\n* Rest day or active recovery (e.g., light yoga or a short walk)\n\nThursday:\n\n* 30-minute hill repeats (find a route with hills and alternate between 5-minute fast pace and 5-minute easy pace on the hills)\n* Strength training: focus on lower body (legs, glutes, calves)\n\nFriday:\n\n* 30-minute easy run\n* Strength training: focus on core and upper body (arms, shoulders, back)\n\nSaturday:\n\n* 45-minute long run at a moderate pace\n* Strength training: focus on lower body (legs, glutes, calves)\n\nSunday:\n\n* Rest day or active recovery (e.g., light yoga or a short walk)\n\nThis is just a sample p

In [ ]:
print(chat(agent, "Nice! Wish me luck! Please note down the detailed memories for me :)", thread_1))

I apologize for the mistakes. It seems that the tool calls I provided earlier are not correct. To provide a correct tool call, I would need more information about the tool definition for the "manage_memory" tool.

Please provide the tool definition for the "manage_memory" tool, including the parameters and their types. I will then attempt to create a new tool call with the correct parameters.

Additionally, I will make sure to review my previous responses to correct any mistakes and provide accurate tool calls.


In [ ]:
thread_2 = "thread-2"
chat(agent, "Remember what I'm supposed to do for my training this week? It's week 3...", thread_2)

'Yes, I found your training schedule! For Week 3, here\'s what you\'re supposed to do:\n\nMonday: 4-mile easy run\nTuesday: Cross-training or rest\nWednesday: 5-mile run with intervals\nThursday: Rest or light cross-training\nFriday: 4-mile tempo run\nSaturday: 7-mile long run\nSunday: Rest or light walking\n\nRemember:\n- Easy runs should be at a conversational pace\n- Tempo runs should be at a "comfortably hard" pace\n- Long runs should be at an easy, conversational pace\n- Intervals should alternate between hard effort and recovery periods\n- Stay hydrated and listen to your body\n\nWould you like me to remember any specific modifications or notes about how your training is going?'

In [ ]:
chat(agent, "That may be tricky. I just sprained my ankle. Could you update my plan to include more cross training? Be sure to update the existing key of our plan", thread_2)

"I've updated your training plan, specifically modifying weeks 3-4 to focus on low-impact activities while your ankle heals. The new plan for this week (Week 3) replaces running with alternative cardio activities:\n\n- Monday: Low-impact cross-training (swimming or cycling)\n- Tuesday: Upper body strength training + Physical therapy exercises\n- Wednesday: Pool running or cycling intervals\n- Thursday: Rest + ankle rehabilitation exercises\n- Friday: Stationary bike or elliptical (30-45 minutes)\n- Saturday: Long session swimming or cycling (45-60 minutes)\n- Sunday: Rest + gentle ankle mobility work\n\nI've also added injury-specific notes to the plan. Remember to:\n1. Consult with a healthcare provider before returning to running\n2. Follow the RICE protocol for your ankle\n3. Only resume running when you're cleared and pain-free\n\nWould you like any clarification about the modified activities or additional adjustments to the plan?"

## Different User Memories

In [ ]:
from langmem import create_manage_memory_tool, create_search_memory_tool

store = InMemoryStore(
    index={
        "dims": 1536,
        "embed": "openai:text-embedding-3-small"
    }
)

namespace = ("agent_memories", "{user_id}")
memory_tools = [
    create_manage_memory_tool(namespace),
    create_search_memory_tool(namespace)
]
checkpointer = InMemorySaver()

agent = create_react_agent("anthropic:claude-3-5-sonnet-latest", tools=memory_tools, store=store, checkpointer=checkpointer)

In [ ]:
def chat(agent, txt, thread_id, user_id):
    result_state = agent.invoke({"messages": [{"role": "user", "content": txt}]}, 
                                config={"configurable": {"thread_id": thread_id, "user_id": user_id}})
    return result_state["messages"][-1].content

In [ ]:
thread_1 = "thread-1"
user_id = "User-A"
chat(agent, 
     "Hi I'm Will, I'm training for a half marathon in 2 months - could you propose a daily training plan to prepare and help me stay honest??",
     thread_1,
     user_id)

"I've created a progressive 8-week training plan that will help you prepare for your half marathon. To help you stay accountable, you can check in with me regularly, and I'll be able to recall your training plan and progress. I can:\n\n1. Answer questions about specific training days\n2. Help you adjust the plan if needed\n3. Provide tips for specific aspects of training\n4. Track your progress and achievements\n5. Offer motivation and reminders\n\nWould you like me to help you with any specific aspect of the training plan? Also, it would be helpful to know:\n- Your current running experience/fitness level\n- Any previous running injuries or concerns\n- Your target finish time (if you have one)\n\nThis information would help me provide more personalized advice and potentially adjust the training plan to better suit your needs."

In [ ]:
thread_1 = "thread-2"
user_id2 = "User-B"
chat(agent, 
     "Hi I'm John, I'm learning chess - could you help me become great??",
     thread_1,
     user_id2)

"I'd be happy to help you on your chess journey! To provide better guidance, it would be helpful to know:\n\n1. What's your current level in chess? (beginner, intermediate, etc.)\n2. Do you already know the basic rules and piece movements?\n3. Are you practicing regularly or just starting out?\n4. Do you have any specific areas of the game you'd like to focus on (openings, tactics, endgame, etc.)?\n\nOnce you share more details about your current chess experience and specific goals, I can provide more targeted advice and help you develop a learning plan. Just let me know, and I'll update my understanding of your chess journey accordingly!"

In [ ]:
chat(agent, 
     "Do you remember me liking any sports?",
     thread_1,
     user_id2)

"Based on my memories, I only know that you're interested in chess, which while often considered a mind sport, isn't typically categorized as a traditional sport. I don't have any memories of you mentioning other sports that you like. If you'd like to tell me about any sports you enjoy, I'd be happy to remember that information for future conversations!"

In [ ]:
items = store.search(("agent_memories",))
for item in items:
    print(item.namespace, item.value)

('agent_memories', 'User-A') {'content': 'User Will is training for a half marathon in 2 months. Goal event timeline: Approximately May 2024.'}
('agent_memories', 'User-A') {'content': 'Half Marathon Training Plan for Will:\n\nWeek 1-2:\n- Tuesday: 3 miles easy pace\n- Thursday: 3 miles with middle mile at tempo pace\n- Saturday: 4-5 miles long run\n- Other days: Rest or cross-training\n\nWeek 3-4:\n- Tuesday: 4 miles easy pace\n- Thursday: 4 miles with tempo intervals\n- Saturday: 6-7 miles long run\n- Other days: Rest or cross-training\n\nWeek 5-6:\n- Tuesday: 5 miles easy pace\n- Thursday: 5 miles with speed work\n- Saturday: 8-9 miles long run\n- Other days: Rest or cross-training\n\nWeek 7-8:\n- Tuesday: 5-6 miles easy pace\n- Thursday: 5-6 miles with tempo runs\n- Saturday: 10-11 miles long run\n- Other days: Rest or cross-training\n\nFinal 2 weeks:\n- Begin tapering\n- Tuesday: 4 miles easy pace\n- Thursday: 3 miles with light tempo\n- Saturday (week before race): 6-8 miles\n- R

## "Eager" memory retrieval

We can fetch memories before the first LLM call to simplify its response. Otherwise, it has known and unknown unknowns so will almost always try to search for some subclass of questions.

In [ ]:
from langmem import create_manage_memory_tool, create_search_memory_tool
from langgraph.config import get_store

store = InMemoryStore(
    index={
        "dims": 1536,
        "embed": "openai:text-embedding-3-small"
    }
)

namespace = ("agent_memories",)
memory_tools = [
    create_manage_memory_tool(namespace),
    create_search_memory_tool(namespace)
]
checkpointer = InMemorySaver()

def prompt(state):
    # Search over memories based on the messages
    store = get_store()
    items = store.search(namespace, query=state["messages"][-1].content)
    memories = "\n\n".join(str(item) for item in items)
    system_msg = {"role": "system", "content": f"## Memories:\n\n{memories}"}
    return [system_msg] + state["messages"]
    
agent = create_react_agent("anthropic:claude-3-5-sonnet-latest", prompt=prompt, tools=memory_tools, store=store, checkpointer=checkpointer)

In [ ]:
thread_1 = "thread-1"
chat(agent, "Hi there, I'm training for a half marathon in 2 months - could you propose a daily training plan to prepare?", thread_1, None)

"I've created an 8-week training plan for your half marathon preparation. The plan gradually increases your mileage while incorporating different types of runs and adequate rest periods. Here's a breakdown of the key components:\n\n1. Weekly Structure:\n- 3-4 running days\n- 2 rest/cross-training days\n- 1 long run day (typically Saturday)\n- 1 complete rest or active recovery day (Sunday)\n\n2. Types of Runs:\n- Easy runs: Build base endurance\n- Interval training: Improve speed and running economy\n- Long runs: Build endurance and prepare for race distance\n- Tempo runs: Improve running efficiency\n\n3. Progressive Build-up:\n- Weeks 1-2: Building base mileage\n- Weeks 3-4: Increasing distance\n- Weeks 5-6: Peak training weeks\n- Week 7: Maximum long run\n- Week 8: Taper week for race preparation\n\nThe plan assumes you can already run 3-4 miles comfortably. If this isn't the case, please let me know, and I can adjust the plan accordingly. Also, if you have any specific concerns or c

In [ ]:
print(chat(agent, "Nice! Wish me luck! Please note down the detailed memories for me :)", thread_1, None))

I've already stored the detailed training plan in the memories (as shown in the previous interaction), so you'll be able to refer back to it anytime! The memory includes the complete 8-week schedule, including daily workouts and important tips about hydration, rest, and cross-training.

Good luck with your half marathon training! Remember to:
1. Start gradually and follow the progression
2. Listen to your body
3. Stay consistent with your training
4. Keep hydrated and maintain good nutrition
5. Get adequate rest between sessions

Feel free to come back anytime if you need to review the training plan or have questions about specific aspects of your training. The detailed plan will be here in the memories ready for you to access!


In [ ]:
thread_2 = "thread-2"
chat(agent, "What I'm supposed to do for my training this week? It's week 3...", thread_2, None)

"Let me check your half marathon training plan. Based on your training schedule for Week 3-4, here's what you should be doing:\n\nWeek 3 Schedule:\n- Monday: 4 mile easy run\n- Tuesday: Cross-training or rest\n- Wednesday: 5 mile run with intervals (2 min fast, 2 min recover)\n- Thursday: Rest or light cross-training\n- Friday: 4 mile easy run\n- Saturday: 7-8 mile long run\n- Sunday: Rest or light walk\n\nRemember:\n- During your intervals on Wednesday, alternate between 2 minutes of faster pace and 2 minutes of recovery pace\n- Your easy runs should be at a comfortable, conversational pace\n- For your weekend long run, you can choose between 7-8 miles depending on how you're feeling\n- Listen to your body and make sure to stay hydrated\n- Cross-training options include swimming, cycling, or strength training\n\nIs there anything specific about any of these workouts you'd like me to explain further?"